In [1]:
from libs.corp_df import *
from tqdm.auto import tqdm
from libs.get_docs import get_pickled_docs as gpd
from libs.dim_red import *
import pandas as pd
import torch
import random
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
# retrieve documents
print('Retrieving Documents...')
cdir = 'corpora/articles_stage2.pickle'
docs = gpd(cdir, verbose=True)

Retrieving Documents...
TIME ELAPSED: 30.63s


In [3]:
vectors = []
labs = []

for doc in tqdm(docs):
    for pg in doc:
        for sent in pg:
            for tok in sent:
                if tok.tokenizer_index != 1:
                    for lem in tok:
                        if lem.pos_tag in ['VB', 'BN', 'BNT']:
                            embed = sum(tok.embedding).numpy()
                            vectors.append(embed)
                            labs.append(lem.shoresh)
                            
df = pd.DataFrame({'lab': labs, 'embedding': vectors})

df['lab_id'] = df['lab'].map({lab: i for lab, i in zip(df['lab'].unique(), range(len(df['lab'].unique())))})

  0%|          | 0/524 [00:00<?, ?it/s]

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

In [5]:
vec_train, vec_test, lab_train, lab_test = train_test_split(df['embedding'].to_list(), df['lab_id'].to_list())

In [8]:
sgd = SGDClassifier()

print('classifying...')
start = time.time()
sgd.fit(vec_train, lab_train)
end = time.time()

print('Training time: {}s'.format(np.round(end-start, 2)))

classifying...
Training time: 100.15s


In [13]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

metrics = [('ACCURACY', accuracy_score), ('F1', f1_score)]

In [24]:
def error_analysis(model, emb_test, lab_test):
    predicted_labs = model.predict(emb_test)
    acc = accuracy_score(lab_test, predicted_labs)
    prec = precision_score(lab_test, predicted_labs, average='weighted')
    rec = recall_score(lab_test, predicted_labs, average='weighted')
    f1 = f1_score(lab_test, predicted_labs, average='weighted')
    
    print(acc, prec, rec, f1)

In [25]:
error_analysis(sgd, vec_test, lab_test)

0.9218177902218393 0.9315901189081436 0.9218177902218393 0.9207710622612838


/home/besalelir1/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/besalelir1/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
